# Step 3: Creating the model inference script
To submit your algorithm to the challenge, you need to create an inference docker container. 

In [1]:
import os
import json
import torch
import creationism
from tqdm import tqdm



from wholeslidedata.interoperability.asap.annotationwriter import write_point_set
from wholeslidedata.image.wholeslideimage import WholeSlideImage
from wholeslidedata.iterators import create_patch_iterator, PatchConfiguration
from wholeslidedata.annotation.labels import Label

from utils.wsdetectron2 import Detectron2DetectionPredictor
from utils.structures import Point

Setting up the paths.

In [2]:
image_path = r'./data/images/pas-cpg/A_P000002_PAS_CPG.tif'
mask_path = r'./data/images/tissue-masks/A_P000002_mask.tif'
output_path = r"./outputs/results"
if not(os.path.isdir(output_path)): os.mkdir (output_path) 
json_filename = "detected-lymphocytes.json"

print(f"Pytorch GPU available: {torch.cuda.is_available()}")
print(image_path, mask_path)

Pytorch GPU available: True
./data/images/pas-cpg/A_P000002_PAS_CPG.tif ./data/images/tissue-masks/A_P000002_mask.tif


Defining patch configuration for each image.

In [3]:
patch_shape=(128, 128,3)
spacings=(0.5,)
overlap=(64,64)
offset=(0,0)
center=False

patch_configuration = PatchConfiguration(patch_shape=patch_shape,
                                         spacings=spacings,
                                         overlap=overlap,
                                         offset=offset,
                                         center=center)

Loading the saved model.

In [4]:
model = Detectron2DetectionPredictor(
    output_dir=output_path,
    threshold= 0.2,
    nms_threshold=0.1,
    weight_root = "./outputs/model_final.pth"
)


/home/jubuntu/anaconda3/envs/monkey/lib/python3.10/site-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_loca

Creating a patch iterator using the roi mask and sliding windows.

In [5]:
iterator = create_patch_iterator(image_path=image_path,
                               mask_path=mask_path,
                               patch_configuration=patch_configuration,
                               cpus=4,
                               backend='asap')

Some useful functions.

In [6]:
def px_to_mm(px: int, spacing: float):
    return px * spacing / 1000

def to_wsd(points):
    """Convert list of coordinates into WSD points"""
    new_points = []
    for i, point in enumerate(points):
        p = Point(
            index=i,
            label=Label("lymphocyted", 1, color="blue"),
            coordinates=[point],
        )
        new_points.append(p)
    return new_points

def write_json_file(*, location, content):
    # Writes a json file
    with open(location, 'w') as f:
        f.write(json.dumps(content, indent=4))

Run inference on an image with loaded model.

In [ ]:
class_thresholds = {
    "lymphocyte": 0.7,  # lymphocyte用のthreshold例
    "monocyte": 0.35     # monocyte用のthreshold例
}

def inference(iterator, predictor, spacing, image_path, output_path, json_filename):
    print("predicting...")
    output_dict = {
        "name": "lymphocytes",
        "type": "Multiple points",
        "version": {"major": 1, "minor": 0},
        "points": [],
    }

    output_dict_monocytes = {
        "name": "monocytes",
        "type": "Multiple points",
        "version": {"major": 1, "minor": 0},
        "points": [],
    }

    output_dict_inflammatory_cells = {
        "name": "inflammatory-cells",
        "type": "Multiple points",
        "version": {"major": 1, "minor": 0},
        "points": [],
    }

    annotations = []
    counter = 0
    
    spacing_min = 0.25
    ratio = spacing/spacing_min
    with WholeSlideImage(image_path) as wsi:
        spacing = wsi.get_real_spacing(spacing_min)


    for x_batch, y_batch, info in tqdm(iterator):
        x_batch = x_batch.squeeze(0)
        y_batch = y_batch.squeeze(0)

        predictions = predictor.predict_on_batch(x_batch)
        for idx, prediction in enumerate(predictions):

            c = info['x']
            r = info['y']

            for detections in prediction:
                x, y, label, confidence = detections.values()
                #print(f'x: {x}, y: {y}, label: {label}, confidence: {confidence}')
                if confidence < class_thresholds[label]:
                    continue
                
                if x == 128 or y == 128:
                    continue

                if y_batch[idx][y][x] == 0:
                    continue
                
                x = x*ratio + c # x is in spacing= 0.5 but c is in spacing = 0.25
                y= y*ratio + r
                prediction_record = {
                    "name" : "Point "+str(counter),
                    "point": [
                        px_to_mm(x, spacing),
                        px_to_mm(y, spacing),
                        0.24199951445730394,
                    ],
                    "probability": confidence,
                }
                
                if label == 'lymphocyte':
                    output_dict["points"].append(prediction_record)
                    output_dict_inflammatory_cells["points"].append(prediction_record)
                elif label == 'monocyte':
                    output_dict_monocytes["points"].append(prediction_record)
                    output_dict_inflammatory_cells["points"].append(prediction_record)
                
                annotations.append((x, y))
                counter += 1

    print(len(output_dict["points"]),len(output_dict_monocytes["points"]),len(output_dict_inflammatory_cells["points"]))  

    print(f"Predicted {len(annotations)} points")
    print("saving predictions...")

    # saving xml file
    annotations_wsd = to_wsd(annotations)
    xml_filename = 'points_results.xml'
    output_path_xml = os.path.join(output_path,xml_filename)
    write_point_set(
        annotations_wsd,
        output_path_xml,
        label_color="blue",
    )

    # saving json file
    output_path_json = os.path.join(output_path, json_filename)
    write_json_file(
        location=output_path_json,
        content=output_dict
    )

    json_filename_monocytes = "detected-monocytes.json"
    # it should be replaced with correct json files
    output_path_json = os.path.join(output_path, json_filename_monocytes)
    write_json_file(
        location=output_path_json,
        content=output_dict_monocytes
    )

    json_filename_inflammatory_cells = "detected-inflammatory-cells.json"
    # it should be replaced with correct json files
    output_path_json = os.path.join(output_path, json_filename_inflammatory_cells)
    write_json_file(
        location=output_path_json,
        content=output_dict_inflammatory_cells
    )

    print("finished!")

In [8]:
inference(
    iterator=iterator,
    predictor=model,
    spacing = spacings[0],
    image_path=image_path,
    output_path=output_path,
    json_filename=json_filename
)

iterator.stop()

predicting...


  0%|          | 0/1506 [00:00<?, ?it/s]

/home/jubuntu/anaconda3/envs/monkey/lib/python3.10/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647382455/work/aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 1506/1506 [00:26<00:00, 57.46it/s]


279 89 368
Predicted 368 points
saving predictions...
finished!


In [9]:
!cp /mnt/hdd2/monkey-challenge/algorithm-tutorial/outputs/results/*.json /mnt/hdd2/monkey-challenge/evaluation/test/input/A_P000002/output/